## Geographical Visualizations - Hate and Crime in USA (1991 - 2018)

### This script contains the following:

1. Import data and libraries
2. Data wrangling
3. Consistency checks
4. Plotting a choropleth

### 1. Import data and libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
#defining path

path = '/Users/Ana/Documents/Hate Crimes in the US'

In [4]:
path

'/Users/Ana/Documents/Hate Crimes in the US'

In [5]:
# Import data
df = pd.read_csv(os.path.join(path, 'Prepared dataset', 'hate_crime_wrangled.csv'))

In [6]:
# Import ".json" file USA
usa_geo = '/Users/Ana/Documents/Hate Crimes in the US/Dataset/us-states.json'

In [7]:
df.head()

Unnamed: 0  INCIDENT_ID  DATA_YEAR        ORI PUB_AGENCY_NAME  \
0           0            2       1991  AZ0072300         Phoenix   
1           1            3       1991  AZ0072300         Phoenix   
2           2            4       1991  AZ0072300         Phoenix   
3           3            5       1991  AZ0072300         Phoenix   
4           4            6       1991  AZ0072300         Phoenix   

  AGENCY_TYPE_NAME STATE_ABBR STATE_NAME DIVISION_NAME REGION_NAME  ...  \
0             City         AZ    Arizona      Mountain        West  ...   
1             City         AZ    Arizona      Mountain        West  ...   
2             City         AZ    Arizona      Mountain        West  ...   
3             City         AZ    Arizona      Mountain        West  ...   
4             City         AZ    Arizona      Mountain        West  ...   

  TOTAL_OFFENDER_COUNT OFFENDER_RACE VICTIM_COUNT  \
0                    0       Unknown            1   
1                    1       Unknown            1   
2                    1       Unknown            1   
3                    1         White            1   
4                    0       Unknown            1   

                               OFFENSE_NAME TOTAL_INDIVIDUAL_VICTIMS  \
0                              Intimidation                      1.0   
1                              Intimidation                      1.0   
2                              Intimidation                      1.0   
3                        Aggravated Assault                      1.0   
4  Destruction/Damage/Vandalism of Property                      0.0   

                        LOCATION_NAME                       BIAS_DESC  \
0                      Residence/Home  Anti-Black or African American   
1      Church/Synagogue/Temple/Mosque                     Anti-Jewish   
2                      Residence/Home                       Anti-Arab   
3  Highway/Road/Alley/Street/Sidewalk  Anti-Black or African American   
4      Church/Synagogue/Temple/Mosque                 Anti-Protestant   

             VICTIM_TYPES MULTIPLE_OFFENSE MULTIPLE_BIAS  
0              Individual                S             S  
1              Individual                S             S  
2              Individual                S             S  
3              Individual                S             S  
4  Religious Organization                S             S  

[5 rows x 23 columns]

In [8]:
df.shape

(201403, 23)

### 2. Data wrangling

In [9]:
# Select only the necessary columns and put them in a list called columns
columns =["DATA_YEAR", "STATE_ABBR", "STATE_NAME", "REGION_NAME", "TOTAL_OFFENDER_COUNT", "OFFENDER_RACE", "VICTIM_COUNT", "OFFENSE_NAME", "TOTAL_INDIVIDUAL_VICTIMS", "BIAS_DESC"]

In [10]:
# Create a subset
hate_crime_visual = df[columns]

In [11]:
hate_crime_visual.head()

DATA_YEAR STATE_ABBR STATE_NAME REGION_NAME  TOTAL_OFFENDER_COUNT  \
0       1991         AZ    Arizona        West                     0   
1       1991         AZ    Arizona        West                     1   
2       1991         AZ    Arizona        West                     1   
3       1991         AZ    Arizona        West                     1   
4       1991         AZ    Arizona        West                     0   

  OFFENDER_RACE  VICTIM_COUNT                              OFFENSE_NAME  \
0       Unknown             1                              Intimidation   
1       Unknown             1                              Intimidation   
2       Unknown             1                              Intimidation   
3         White             1                        Aggravated Assault   
4       Unknown             1  Destruction/Damage/Vandalism of Property   

   TOTAL_INDIVIDUAL_VICTIMS                       BIAS_DESC  
0                       1.0  Anti-Black or African American  
1                       1.0                     Anti-Jewish  
2                       1.0                       Anti-Arab  
3                       1.0  Anti-Black or African American  
4                       0.0                 Anti-Protestant

### 3. Comsistency checks

In [12]:
# Check for missing values 
hate_crime_visual.isnull().sum()

DATA_YEAR                      0
STATE_ABBR                     0
STATE_NAME                     0
REGION_NAME                    0
TOTAL_OFFENDER_COUNT           0
OFFENDER_RACE                 19
VICTIM_COUNT                   0
OFFENSE_NAME                   0
TOTAL_INDIVIDUAL_VICTIMS    1606
BIAS_DESC                      0
dtype: int64

In [13]:
hate_crime_visual['OFFENDER_RACE'].fillna('Unknown',inplace=True)
hate_crime_visual['TOTAL_INDIVIDUAL_VICTIMS'].fillna('Unknown',inplace=True)

/var/folders/s5/l8yzj1zj6cb2wnts0l2khjl40000gq/T/ipykernel_21401/3450558985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_crime_visual['OFFENDER_RACE'].fillna('Unknown',inplace=True)
/var/folders/s5/l8yzj1zj6cb2wnts0l2khjl40000gq/T/ipykernel_21401/3450558985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_crime_visual['TOTAL_INDIVIDUAL_VICTIMS'].fillna('Unknown',inplace=True)


In [14]:
# recheck for missing values 
hate_crime_visual.isnull().sum()

DATA_YEAR                   0
STATE_ABBR                  0
STATE_NAME                  0
REGION_NAME                 0
TOTAL_OFFENDER_COUNT        0
OFFENDER_RACE               0
VICTIM_COUNT                0
OFFENSE_NAME                0
TOTAL_INDIVIDUAL_VICTIMS    0
BIAS_DESC                   0
dtype: int64

In [15]:
# Checking for duplicates
dups = hate_crime_visual.duplicated()

In [16]:
dups.shape # no dups

(201403,)

In [22]:
# Hate Crime Biases Visuals  (only showing data for the big racial/identity groups)

hate_crime_visuals = hate_crime_visual[(hate_crime_visual['BIAS_DESC'] == 'Anti-American Indian or Alaska Native') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Arab') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Asian') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Black or African American') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Hispanic or Latino') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Other Race/Ethnicity/Ancestry')
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Gay (Male)') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Lesbian(Female)')
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Lesbian, Gay, Bisexual, or Transgender (Mixed Group)')                              
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Jewish') 
         | (hate_crime_visual['BIAS_DESC'] == 'Anti-Islamic')]

In [23]:
hate_crime_visuals.head()

DATA_YEAR STATE_ABBR STATE_NAME REGION_NAME  TOTAL_OFFENDER_COUNT  \
0       1991         AZ    Arizona        West                     0   
1       1991         AZ    Arizona        West                     1   
2       1991         AZ    Arizona        West                     1   
3       1991         AZ    Arizona        West                     1   
5       1991         AZ    Arizona        West                     1   

  OFFENDER_RACE  VICTIM_COUNT        OFFENSE_NAME TOTAL_INDIVIDUAL_VICTIMS  \
0       Unknown             1        Intimidation                      1.0   
1       Unknown             1        Intimidation                      1.0   
2       Unknown             1        Intimidation                      1.0   
3         White             1  Aggravated Assault                      1.0   
5       Unknown             1        Intimidation                      1.0   

                        BIAS_DESC  
0  Anti-Black or African American  
1                     Anti-Jewish  
2                       Anti-Arab  
3  Anti-Black or African American  
5  Anti-Black or African American

In [25]:
def nb_check(abbr):
    if abbr == 'NB':
        return 'NE'
    else:
        return abbr

In [26]:
by_state = hate_crime_visuals
by_state['STATE_ABBR'] = by_state['STATE_ABBR'].apply(nb_check)

/var/folders/s5/l8yzj1zj6cb2wnts0l2khjl40000gq/T/ipykernel_21401/2590119508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  by_state['STATE_ABBR'] = by_state['STATE_ABBR'].apply(nb_check)


In [27]:
by_state[by_state['STATE_ABBR']=='NE']

DATA_YEAR STATE_ABBR STATE_NAME REGION_NAME  TOTAL_OFFENDER_COUNT  \
35757        1996         NE   Nebraska     Midwest                     1   
35758        1996         NE   Nebraska     Midwest                     1   
35759        1996         NE   Nebraska     Midwest                     1   
44353        1997         NE   Nebraska     Midwest                     0   
44354        1997         NE   Nebraska     Midwest                     4   
...           ...        ...        ...         ...                   ...   
198735       2018         NE   Nebraska     Midwest                     0   
198737       2017         NE   Nebraska     Midwest                     1   
198738       2017         NE   Nebraska     Midwest                     0   
198739       2017         NE   Nebraska     Midwest                     1   
198740       2018         NE   Nebraska     Midwest                     0   

                    OFFENDER_RACE  VICTIM_COUNT  \
35757                       White             1   
35758   Black or African American             1   
35759   Black or African American             1   
44353                     Unknown             1   
44354                       White             4   
...                           ...           ...   
198735                    Unknown             1   
198737  Black or African American             1   
198738                    Unknown             1   
198739                      White             1   
198740                    Unknown             1   

                                    OFFENSE_NAME TOTAL_INDIVIDUAL_VICTIMS  \
35757                             Simple Assault                      1.0   
35758                         Aggravated Assault                      1.0   
35759                         Aggravated Assault                      1.0   
44353   Destruction/Damage/Vandalism of Property                      1.0   
44354                         Aggravated Assault                      4.0   
...                                          ...                      ...   
198735  Destruction/Damage/Vandalism of Property                      0.0   
198737                        Aggravated Assault                      1.0   
198738                              Intimidation                      1.0   
198739                              Intimidation                      1.0   
198740                            Simple Assault                      1.0   

                                                BIAS_DESC  
35757                             Anti-Hispanic or Latino  
35758                      Anti-Black or African American  
35759                      Anti-Black or African American  
44353                                         Anti-Jewish  
44354                  Anti-Other Race/Ethnicity/Ancestry  
...                                                   ...  
198735  Anti-Lesbian, Gay, Bisexual, or Transgender (M...  
198737                     Anti-Black or African American  
198738                                    Anti-Gay (Male)  
198739                     Anti-Black or African American  
198740  Anti-Lesbian, Gay, Bisexual, or Transgender (M...  

[745 rows x 10 columns]

In [28]:
by_state = hate_crime_visuals.groupby('STATE_NAME').count()

In [29]:
by_state.head()

DATA_YEAR  STATE_ABBR  REGION_NAME  TOTAL_OFFENDER_COUNT  \
STATE_NAME                                                             
Alabama           149         149          149                   149   
Alaska            167         167          167                   167   
Arizona          4905        4905         4905                  4905   
Arkansas          550         550          550                   550   
California      27650       27650        27650                 27650   

            OFFENDER_RACE  VICTIM_COUNT  OFFENSE_NAME  \
STATE_NAME                                              
Alabama               149           149           149   
Alaska                167           167           167   
Arizona              4905          4905          4905   
Arkansas              550           550           550   
California          27650         27650         27650   

            TOTAL_INDIVIDUAL_VICTIMS  BIAS_DESC  
STATE_NAME                                       
Alabama                          149        149  
Alaska                           167        167  
Arizona                         4905       4905  
Arkansas                         550        550  
California                     27650      27650

### 4. Plotting a choropleth 

In [31]:
by_state = by_state.reset_index()

In [32]:
# Create a data frame with just the states and the values we want plotted

data_to_plot = by_state[['STATE_NAME','TOTAL_INDIVIDUAL_VICTIMS']]
data_to_plot.head()

STATE_NAME  TOTAL_INDIVIDUAL_VICTIMS
0     Alabama                       149
1      Alaska                       167
2     Arizona                      4905
3    Arkansas                       550
4  California                     27650

In [43]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = usa_geo, 
    data = data_to_plot,
    columns = ['STATE_NAME', 'TOTAL_INDIVIDUAL_VICTIMS'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'PuRd', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Victims of Hate Crime").add_to(map)
folium.LayerControl().add_to(map)

map

#### The above  clearly demonstrates that, of all the states, California is the one with the highest total reported hate crimes from the years 1991-2018

In [44]:
map.save('plot_data.html')

In [45]:
#exporting data
hate_crime_visuals.to_csv(os.path.join(path, 'Prepared dataset', 'hate_crimes_wrangled_2.csv'))